In [1]:
# Propogate test with cora (to make sure it works)

In [5]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn

In [6]:
N = ...  # Num nodes
f = ...  # Num input feats
h = 32  # Hidden state for node size
m = 32  # Message size
num_class = 8
o = num_class  # Output size
avg_func = lambda X: X.mean(axis=0)
num_props = 3
num_epochs = 1

Input I need:
1. The graph (cora is undirected so I will just make each edge a 2-way) in networkx
2. predecessors list since this is a static graph and i need it for the aggregate
3. N x h that is updated every prop because its a static graph so this is faster

In [7]:
# Make graph

In [8]:
# Make numpy matrix of N x f
node_feats = np.zeros(shape=(N, f))

TypeError: 'ellipsis' object cannot be interpreted as an integer

In [9]:
predecessors = []
for node in G.nodes:
    predecessors.append(list(G.predecessors(node)))
assert len(predecessors) == N

NameError: name 'G' is not defined

In [10]:
# Input: (N x h) which is all nodes hidden states
# Outut: (N x m) all nodes messages
class MessageModel:
    def __init__(self):
        self.model = self._make_model()
        
    def _make_model(self):
        input_layer = Input(shape=(h,))
        output_layer = Dense(m, activation='relu')(input_layer)
        model = Model(input_layer, output_layer)
        return model
    
    def get_messages(self, nodes):
        assert nodes.shape == (N, h)
        messages = self.model.predict(nodes)
        assert messages.shape == (N, m)
        return messages

In [11]:
# Input: (N x m) messages
# Output: (N x m)  aggregated messages
def aggregate(predecessors, messages, agg_func=avg_func):
    agg = []
    # Collect all in predecessors for each node, if a node has no preds then just 0s for it
    for preds in predecessors:
        if len(preds) > 0:
            in_mess = messages[preds, :]
            assert in_mess.shape == (len(preds), m) or in_mess.shape == (m,)  # if one in-node
            agg_in_mess = agg_func(in_mess)
            assert agg_in_mess.shape == (m,)
            agg.append(agg_in_mess)
        else:
            agg.append(np.zeros(shape=(m,)))
    # Stack
    stacked = np.stack(agg)
    assert stacked.shape == (N, m)
    return stacked

In [12]:
# Input: (N, m) agg messages
# Output: (N, h) new hidden states for nodes
class UpdateModel:
    def __init__(self):
        self.model = self._make_model()
        
    def _make_model(self):
        input_layer = Input(shape=(m,))
        output_layer = Dense(h, activation='relu')(input_layer)
        model = Model(input_layer, output_layer)
        return model
    
    def get_updates(self, messages):
        assert messages.shape == (N, m)
        updates = self.model.predict(messages)
        assert updates.shape == (N, h)
        return updates

In [13]:
# Input: (N, h)  updated node hidden states
# Output: (N, o)  outputs for each node (softmax on classes)
class OutputModel:
    def __init__(self):
        self.model = self._make_model()
    
    def _make_model(self):
        input_layer = Input(shape=(h,))
        output_layer = Dense(o, activation='softmax')(input_layer)
        model = Model(input_layer, output_layer)
        return model
    
    def get_outputs(self, nodes):
        assert nodes.shape == (N, h)
        outputs = self.model.predict(nodes)
        assert outputs.shape == (N, o)
        return outputs

In [23]:
# Make objects
message_M = MessageModel()
update_M = UpdateModel()
output_M = OutputModel()

In [27]:
def propogate(node_states, get_output):
    messages = message_M.get_messages(node_states)
    aggregates = aggregate(predecessors, messages)
    updates = update_M.get_updates(aggregates)
    if get_output:
        outputs = output_M.get_messages(updates)
        return updates, outputs
    return updates, None

In [28]:
def run_epoch(node_feats):
    node_states = node_feats.copy()
    for p in range(num_props):
        node_states, outputs = propogate(node_states, p == num_props-1)
    # Train

In [ ]:
for epoch in range(num_epochs):
    run_epoch()